## Load Data

In [1]:
!pip install pyarrow
!pip install catboost

     |████████████████████████████████| 65.7MB 69kB/s 


In [2]:
import numpy as np
import pandas as pd 
import gc
import xgboost as xgb
import lightgbm as lgb
import catboost as cat
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [3]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [4]:
train_raw = pd.read_parquet("./drive/MyDrive/Colab Notebooks/data/feature_u_m_um_b_c_mb_mc", engine="pyarrow")

In [5]:
train_raw.head(10)

,merchant_id,cat_id,brand_id,user_id,label,gender,age,item_id,u_ttl_clk_atc_cnt,u_ttl_buy_cnt,u_ttl_atf_cnt,u_clk_atc_cnt_m5,u_clk_atc_cnt_m6,u_clk_atc_cnt_m7,u_clk_atc_cnt_m8,u_clk_atc_cnt_m9,u_clk_atc_cnt_m10,u_clk_atc_cnt_m11,u_buy_cnt_m5,u_buy_cnt_m6,u_buy_cnt_m7,u_buy_cnt_m8,u_buy_cnt_m9,u_buy_cnt_m10,u_buy_cnt_m11,u_atf_cnt_m5,u_atf_cnt_m6,u_atf_cnt_m7,u_atf_cnt_m8,u_atf_cnt_m9,u_atf_cnt_m10,u_atf_cnt_m11,u_ttl_clk_atc_ratio,u_ttl_buy_ratio,u_ttl_atf_ratio,u_clk_atc_ratio_m5,u_clk_atc_ratio_m6,u_clk_atc_ratio_m7,u_clk_atc_ratio_m8,u_clk_atc_ratio_m9,u_clk_atc_ratio_m10,u_clk_atc_ratio_m11,u_buy_ratio_m5,u_buy_ratio_m6,u_buy_ratio_m7,u_buy_ratio_m8,u_buy_ratio_m9,u_buy_ratio_m10,u_buy_ratio_m11,u_atf_ratio_m5,u_atf_ratio_m6,u_atf_ratio_m7,u_atf_ratio_m8,u_atf_ratio_m9,u_atf_ratio_m10,u_atf_ratio_m11,u_clk_atc_day_cnt,u_buy_day_cnt,u_atf_day_cnt,u_clk_atc_day_cnt_m5,u_clk_atc_day_cnt_m6,u_clk_atc_day_cnt_m7,u_clk_atc_day_cnt_m8,u_clk_atc_day_cnt_m9,u_clk_atc_day_cnt_m10,u_clk_atc_day_cnt_m11,u_buy_day_cnt_m5,u_buy_day_cnt_m6,u_buy_day_cnt_m7,u_buy_day_cnt_m8,u_buy_day_cnt_m9,u_buy_day_cnt_m10,u_buy_day_cnt_m11,u_atf_day_cnt_m5,u_atf_day_cnt_m6,u_atf_day_cnt_m7,u_atf_day_cnt_m8,u_atf_day_cnt_m9,u_atf_day_cnt_m10,u_atf_day_cnt_m11,u_clk_atc_item_div,u_buy_item_div,u_atf_item_div,u_clk_atc_cat_div,u_buy_cat_div,u_atf_cat_div,u_clk_atc_brd_div,u_buy_brd_div,u_atf_brd_div,u_clk_atc_mer_div,u_buy_mer_div,u_atf_mer_div,u_clk_atc_mean,u_buy_mean,u_atf_mean,u_clk_atc_median,u_buy_median,u_atf_median,u_clk_atc_std,u_buy_std,u_atf_std,u_clk_atc_max,u_buy_max,u_atf_max,u_clk_atc_day_cnt_mean,u_clk_atc_day_cnt_max,u_clk_atc_day_cnt_std,u_clk_atc_day_cnt_med,u_buy_day_cnt_mean,u_buy_day_cnt_max,u_buy_day_cnt_std,u_buy_day_cnt_med,u_atf_day_cnt_mean,u_atf_day_cnt_max,u_atf_day_cnt_std,u_atf_day_cnt_med,u_clk_atc_item_cnt_mean,u_clk_atc_item_cnt_max,u_clk_atc_item_cnt_std,u_clk_atc_item_cnt_med,u_buy_item_cnt_mean,u_buy_item_cnt_max,u_buy_item_cnt_std,u_buy_item_cnt_med,u_atf_item_cnt_mean,u_atf_item_cnt_max,u_atf_item_cnt_std,u_atf_item_cnt_med,u_clk_atc_cnt_double11,u_buy_cnt_double11,u_atf_cnt_double11,u_clk_atc_cnt_1wpre_double11,u_buy_cnt_1wpre_double11,u_atf_cnt_1wpre_double11,u_clk_atc_ratio_double11,u_clk_atc_ratio_1wpre_double11,u_buy_ratio_double11,u_buy_ratio_1wpre_double11,u_atf_ratio_double11,u_atf_ratio_1wpre_double11,m_ttl_clk_atc_cnt,m_ttl_buy_cnt,m_ttl_atf_cnt,m_clk_atc_cnt_m5,m_clk_atc_cnt_m6,m_clk_atc_cnt_m7,m_clk_atc_cnt_m8,m_clk_atc_cnt_m9,m_clk_atc_cnt_m10,m_clk_atc_cnt_m11,m_buy_cnt_m5,m_buy_cnt_m6,m_buy_cnt_m7,m_buy_cnt_m8,m_buy_cnt_m9,m_buy_cnt_m10,m_buy_cnt_m11,m_atf_cnt_m5,m_atf_cnt_m6,m_atf_cnt_m7,m_atf_cnt_m8,m_atf_cnt_m9,m_atf_cnt_m10,m_atf_cnt_m11,m_ttl_clk_atc_ratio,m_ttl_buy_ratio,m_ttl_atf_ratio,m_clk_atc_ratio_m5,m_clk_atc_ratio_m6,m_clk_atc_ratio_m7,m_clk_atc_ratio_m8,m_clk_atc_ratio_m9,m_clk_atc_ratio_m10,m_clk_atc_ratio_m11,m_buy_ratio_m5,m_buy_ratio_m6,m_buy_ratio_m7,m_buy_ratio_m8,m_buy_ratio_m9,m_buy_ratio_m10,m_buy_ratio_m11,m_atf_ratio_m5,m_atf_ratio_m6,m_atf_ratio_m7,m_atf_ratio_m8,m_atf_ratio_m9,m_atf_ratio_m10,m_atf_ratio_m11,m_clk_atc_day_cnt,m_buy_day_cnt,m_atf_day_cnt,m_clk_atc_day_cnt_m5,m_clk_atc_day_cnt_m6,m_clk_atc_day_cnt_m7,m_clk_atc_day_cnt_m8,m_clk_atc_day_cnt_m9,m_clk_atc_day_cnt_m10,m_clk_atc_day_cnt_m11,m_buy_day_cnt_m5,m_buy_day_cnt_m6,m_buy_day_cnt_m7,m_buy_day_cnt_m8,m_buy_day_cnt_m9,m_buy_day_cnt_m10,m_buy_day_cnt_m11,m_atf_day_cnt_m5,m_atf_day_cnt_m6,m_atf_day_cnt_m7,m_atf_day_cnt_m8,m_atf_day_cnt_m9,m_atf_day_cnt_m10,m_atf_day_cnt_m11,m_clk_atc_item_div,m_buy_item_div,m_atf_item_div,m_clk_atc_cat_div,m_buy_cat_div,m_atf_cat_div,m_clk_atc_brd_div,m_buy_brd_div,m_atf_brd_div,m_clk_atc_mer_div,m_buy_mer_div,m_atf_mer_div,m_clk_atc_mean,m_buy_mean,m_atf_mean,m_clk_atc_median,m_buy_median,m_atf_median,m_clk_atc_std,m_buy_std,m_atf_std,m_clk_atc_max,m_buy_max,m_atf_max,m_clk_atc_day_cnt_mean,m_clk_atc_day_cnt_max,m_clk_atc_day_cnt_std,m_clk_atc_day_cnt_med,m_buy_day_cnt_mean,m_bu

In [6]:
train = train_raw.dropna()
del train_raw
gc.collect()

13

In [7]:
train["gender"] = train["gender"].astype("int8")
train["age"] = train["age"].astype("int8")
train["m_popular_age"] = train["m_popular_age"].astype("int8")
train["m_popular_gender"] = train["m_popular_gender"].astype("int8")

## Feature Selection

### Nearly Zero Variance

In [8]:
train_X, train_y = train.drop(["user_id", "merchant_id", "label", "item_id", "cat_id", "brand_id"], axis=1), train["label"]

In [9]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import cross_val_score

var_thres = VarianceThreshold(threshold=0.05)
var_thres.fit(train_X)

# 交叉验证
train_X_varthresh = train_X.iloc[:,var_thres.get_support(indices=True)]
print("使用方差阈值筛选出来的特征结果索引:",var_thres.get_support(indices=True))

使用方差阈值筛选出来的特征结果索引: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  29  30  31  32  33  34  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 103 104 106 107 108 109 110
 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128
 129 130 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149
 150 151 152 153 154 155 156 157 182 183 184 185 186 187 188 189 190 191
 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209
 210 211 212 213 214 218 219 220 221 222 223 224 225 226 227 228 229 230
 231 232 235 239 240 242 243 244 245 246 247 248 250 251 252 254 255 256
 257 258 259 266 267 268 269 270 271 272 273 274 275 276 277 278 279 286
 290 291 292 293 294 295 302 303 310 318 320 322 323 324 325 326 327 340
 341 342 343 344 345 346 347 348

## Split the training and testing data

In [ ]:
train_X, train_y = train_X_varthresh, train['label']

X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=.2)

### CatBoost

In [ ]:
def cat_train(X_train, y_train, X_valid, y_valid, verbose=True):
    model_cat = cat.CatBoostClassifier(learning_rate=0.02, 
                                       iterations=5000, 
                                       eval_metric='AUC', 
                                       od_wait=50,
                                       od_type='Iter', 
                                       random_state=10, 
                                       thread_count=8, 
                                       l2_leaf_reg=1, 
                                       verbose=verbose)
    
    model_cat.fit(X_train, 
                  y_train, 
                  eval_set=[(X_valid, y_valid)], 
                  early_stopping_rounds=50,
                  use_best_model=True)

    print(model_cat.best_score_['validation']['AUC'])
    return model_cat

In [ ]:
%%time
model_cat = cat_train(X_train, y_train, X_valid, y_valid, verbose=False)

0.6887060633108057
CPU times: user 12min 36s, sys: 39.4 s, total: 13min 16s
Wall time: 6min 50s


## Grid Search

In [ ]:
train_X, train_y = train_X_varthresh, train['label']

In [ ]:
param_test_1 = {"iterations": [100, 300, 500],
              "max_depth": [8, 10, 12]}

cat_model = cat.CatBoostClassifier(learning_rate=0.1, 
                                       iterations=5000, 
                                       max_depth=8,
                                       eval_metric='AUC', 
                                       od_wait=50,
                                       od_type='Iter', 
                                       random_state=10, 
                                       thread_count=8, 
                                       l2_leaf_reg=1)

CATsearch_1 = GridSearchCV(estimator=cat_model,
                           param_grid=param_test_1,
                           scoring="roc_auc",
                           cv=5, verbose=1, n_jobs=1)

CATsearch_1.fit(train_X, train_y)

CATsearch_1.cv_results_, CATsearch_1.best_params_, CATsearch_1.best_score_

In [ ]:
param_test_2 = {"learning_rate": [0.01, 0.02, 0.03, 0.05, 0.1, 0.2]}

cat_model = cat.CatBoostClassifier(learning_rate=0.1, 
                                       iterations=5000, 
                                       max_depth=8,
                                       eval_metric='AUC', 
                                       od_wait=50,
                                       od_type='Iter', 
                                       random_state=10, 
                                       thread_count=8, 
                                       l2_leaf_reg=1, 
                                       task_type='GPU',
                                       verbose=False)

CATsearch_2 = GridSearchCV(estimator=cat_model,
                           param_grid=param_test_2,
                           scoring="roc_auc",
                           cv=5, verbose=1, n_jobs=1)

CATsearch_2.fit(train_X, train_y)

CATsearch_2.cv_results_, CATsearch_2.best_params_, CATsearch_2.best_score_

## K-fold

In [10]:
def get_train_testDF(train_df,label_df):
    skv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    trainX = []
    trainY = []
    testX = []
    testY = []
    for train_index, test_index in skv.split(X=train_df, y=label_df):
        train_x, train_y, test_x, test_y = train_df.iloc[train_index, :], label_df.iloc[train_index], \
                                            train_df.iloc[test_index, :], label_df.iloc[test_index]

        trainX.append(train_x)
        trainY.append(train_y)
        testX.append(test_x)
        testY.append(test_y)
    return trainX, testX, trainY, testY

In [11]:
train_X, train_y = train_X_varthresh, train['label']

X_train, X_valid, y_train, y_valid = get_train_testDF(train_X, train_y)

In [13]:
# catgbm模型
# lr=0.1, iterations=100, max_depth=8
%%time
scores = []
for i in range(5):
    print('\n============================CAT training use Data {}/5============================\n'.format(i+1))
    model_cat = cat.CatBoostClassifier(learning_rate=0.02, 
                                       iterations=5000, 
                                       max_depth=6,
                                       eval_metric='AUC', 
                                       od_wait=50,
                                       od_type='Iter', 
                                       random_state=10, 
                                       thread_count=8, 
                                       l2_leaf_reg=1, 
                                       task_type='GPU',
                                       verbose=False)
    model_cat.fit(X_train[i], y_train[i], eval_set=[(X_valid[i], y_valid[i])], early_stopping_rounds=50, use_best_model=True)
    # print(model_cat.evals_result_)
    print(model_cat.best_score_['validation']['AUC'])
    scores.append(model_cat.best_score_['validation']['AUC'])
print("AUC: %0.3f (+/- %0.5f)" % (np.mean(scores), np.std(scores)))


============================CAT training use Data 1/5============================

0.6894829273223877

============================CAT training use Data 2/5============================

0.694005936384201

============================CAT training use Data 3/5============================

0.6923162043094635

============================CAT training use Data 4/5============================

0.6839672327041626

============================CAT training use Data 5/5============================

0.6911348104476929
AUC: 0.690 (+/- 0.00344)
CPU times: user 2min 56s, sys: 2min 17s, total: 5min 14s
Wall time: 3min 1s


In [ ]:
# catgbm模型
# lr=0.1, iterations=500, max_depth=8
%%time
pred_cats = []
for i in range(5):
    print('\n============================CAT training use Data {}/5============================\n'.format(i+1))
    model_cat = cat.CatBoostClassifier(learning_rate=0.1, 
                                       iterations=500, 
                                       max_depth=8,
                                       eval_metric='AUC', 
                                       od_wait=50,
                                       od_type='Iter', 
                                       random_state=10, 
                                       thread_count=8, 
                                       l2_leaf_reg=1, 
                                       task_type='GPU',
                                       verbose=False)
    model_cat.fit(X_train[i], y_train[i], eval_set=[(X_valid[i], y_valid[i])], early_stopping_rounds=50, use_best_model=True)
    # print(model_cat.evals_result_)
    print(model_cat.best_score_['validation']['AUC'])


============================CAT training use Data 1/5============================

0.6844939887523651

============================CAT training use Data 2/5============================

0.6864968538284302

============================CAT training use Data 3/5============================

0.6904087662696838

============================CAT training use Data 4/5============================

0.6798604726791382

============================CAT training use Data 5/5============================

0.6850922405719757
CPU times: user 1min 12s, sys: 35.3 s, total: 1min 47s
Wall time: 57.3 s


In [ ]:
# catgbm模型
# lr=0.1, iterations=1000, max_depth=8
%%time
pred_cats = []
for i in range(5):
    print('\n============================CAT training use Data {}/5============================\n'.format(i+1))
    model_cat = cat.CatBoostClassifier(learning_rate=0.1, 
                                       iterations=1000, 
                                       max_depth=8,
                                       eval_metric='AUC', 
                                       od_wait=50,
                                       od_type='Iter', 
                                       random_state=10, 
                                       thread_count=8, 
                                       l2_leaf_reg=1, 
                                       task_type='GPU',
                                       verbose=False)
    model_cat.fit(X_train[i], y_train[i], eval_set=[(X_valid[i], y_valid[i])], early_stopping_rounds=50, use_best_model=True)
    # print(model_cat.evals_result_)
    print(model_cat.best_score_['validation']['AUC'])


============================CAT training use Data 1/5============================

0.6844940483570099

============================CAT training use Data 2/5============================

0.6864968538284302

============================CAT training use Data 3/5============================

0.6904087960720062

============================CAT training use Data 4/5============================

0.679860532283783

============================CAT training use Data 5/5============================

0.6850922405719757
CPU times: user 1min 10s, sys: 37.2 s, total: 1min 48s
Wall time: 57.7 s


In [ ]:
# catgbm模型
# lr=0.1, iterations=1500, max_depth=8
%%time
pred_cats = []
for i in range(5):
    print('\n============================CAT training use Data {}/5============================\n'.format(i+1))
    model_cat = cat.CatBoostClassifier(learning_rate=0.1, 
                                       iterations=1500, 
                                       max_depth=8,
                                       eval_metric='AUC', 
                                       od_wait=50,
                                       od_type='Iter', 
                                       random_state=10, 
                                       thread_count=8, 
                                       l2_leaf_reg=1, 
                                       task_type='GPU',
                                       verbose=False)
    model_cat.fit(X_train[i], y_train[i], eval_set=[(X_valid[i], y_valid[i])], early_stopping_rounds=50, use_best_model=True)
    # print(model_cat.evals_result_)
    print(model_cat.best_score_['validation']['AUC'])


============================CAT training use Data 1/5============================

0.6844940483570099

============================CAT training use Data 2/5============================

0.6864968836307526

============================CAT training use Data 3/5============================

0.690408855676651

============================CAT training use Data 4/5============================

0.6798604726791382

============================CAT training use Data 5/5============================

0.6850922703742981
CPU times: user 1min 11s, sys: 36.4 s, total: 1min 48s
Wall time: 57.6 s


In [ ]:
# catgbm模型
# lr=0.1, iterations=100, max_depth=10
%%time
pred_cats = []
for i in range(5):
    print('\n============================CAT training use Data {}/5============================\n'.format(i+1))
    model_cat = cat.CatBoostClassifier(learning_rate=0.1, 
                                       iterations=100, 
                                       max_depth=10,
                                       eval_metric='AUC', 
                                       od_wait=50,
                                       od_type='Iter', 
                                       random_state=10, 
                                       thread_count=8, 
                                       l2_leaf_reg=1, 
                                       task_type='GPU',
                                       verbose=False)
    model_cat.fit(X_train[i], y_train[i], eval_set=[(X_valid[i], y_valid[i])], early_stopping_rounds=50, use_best_model=True)
    # print(model_cat.evals_result_)
    print(model_cat.best_score_['validation']['AUC'])


============================CAT training use Data 1/5============================

0.6787307858467102

============================CAT training use Data 2/5============================

0.6837489306926727

============================CAT training use Data 3/5============================

0.6802982091903687

============================CAT training use Data 4/5============================

0.6702332496643066

============================CAT training use Data 5/5============================

0.6788195371627808
CPU times: user 1min 5s, sys: 30.9 s, total: 1min 36s
Wall time: 50.4 s


In [ ]:
# catgbm模型
# lr=0.1, iterations=500, max_depth=10
%%time
pred_cats = []
for i in range(5):
    print('\n============================CAT training use Data {}/5============================\n'.format(i+1))
    model_cat = cat.CatBoostClassifier(learning_rate=0.1, 
                                       iterations=500, 
                                       max_depth=10,
                                       eval_metric='AUC', 
                                       od_wait=50,
                                       od_type='Iter', 
                                       random_state=10, 
                                       thread_count=8, 
                                       l2_leaf_reg=1, 
                                       task_type='GPU',
                                       verbose=False)
    model_cat.fit(X_train[i], y_train[i], eval_set=[(X_valid[i], y_valid[i])], early_stopping_rounds=50, use_best_model=True)
    # print(model_cat.evals_result_)
    print(model_cat.best_score_['validation']['AUC'])


============================CAT training use Data 1/5============================

0.6793952286243439

============================CAT training use Data 2/5============================

0.6837488114833832

============================CAT training use Data 3/5============================

0.6802981495857239

============================CAT training use Data 4/5============================

0.6703081429004669

============================CAT training use Data 5/5============================

0.6788195371627808
CPU times: user 1min 12s, sys: 42.3 s, total: 1min 54s
Wall time: 1min


In [ ]:
# catgbm模型
# lr=0.1, iterations=1000, max_depth=10
%%time
pred_cats = []
for i in range(5):
    print('\n============================CAT training use Data {}/5============================\n'.format(i+1))
    model_cat = cat.CatBoostClassifier(learning_rate=0.1, 
                                       iterations=1000, 
                                       max_depth=10,
                                       eval_metric='AUC', 
                                       od_wait=50,
                                       od_type='Iter', 
                                       random_state=10, 
                                       thread_count=8, 
                                       l2_leaf_reg=1, 
                                       task_type='GPU',
                                       verbose=False)
    model_cat.fit(X_train[i], y_train[i], eval_set=[(X_valid[i], y_valid[i])], early_stopping_rounds=50, use_best_model=True)
    # print(model_cat.evals_result_)
    print(model_cat.best_score_['validation']['AUC'])


============================CAT training use Data 1/5============================

0.6793952286243439

============================CAT training use Data 2/5============================

0.6837489604949951

============================CAT training use Data 3/5============================

0.6802981793880463

============================CAT training use Data 4/5============================

0.6703081130981445

============================CAT training use Data 5/5============================

0.6788195371627808
CPU times: user 1min 12s, sys: 41.5 s, total: 1min 53s
Wall time: 59.7 s


In [ ]:
# catgbm模型
# lr=0.1, iterations=1500, max_depth=10
%%time
pred_cats = []
for i in range(5):
    print('\n============================CAT training use Data {}/5============================\n'.format(i+1))
    model_cat = cat.CatBoostClassifier(learning_rate=0.1, 
                                       iterations=1500, 
                                       max_depth=10,
                                       eval_metric='AUC', 
                                       od_wait=50,
                                       od_type='Iter', 
                                       random_state=10, 
                                       thread_count=8, 
                                       l2_leaf_reg=1, 
                                       task_type='GPU',
                                       verbose=False)
    model_cat.fit(X_train[i], y_train[i], eval_set=[(X_valid[i], y_valid[i])], early_stopping_rounds=50, use_best_model=True)
    # print(model_cat.evals_result_)
    print(model_cat.best_score_['validation']['AUC'])


============================CAT training use Data 1/5============================

0.6793952286243439

============================CAT training use Data 2/5============================

0.6837489604949951

============================CAT training use Data 3/5============================

0.6802981793880463

============================CAT training use Data 4/5============================

0.6703081429004669

============================CAT training use Data 5/5============================

0.6797128617763519
CPU times: user 1min 14s, sys: 45.4 s, total: 1min 59s
Wall time: 1min 2s


In [ ]:
# catgbm模型
# lr=0.1, iterations=100, max_depth=12
%%time
pred_cats = []
for i in range(5):
    print('\n============================CAT training use Data {}/5============================\n'.format(i+1))
    model_cat = cat.CatBoostClassifier(learning_rate=0.1, 
                                       iterations=100, 
                                       max_depth=12,
                                       eval_metric='AUC', 
                                       od_wait=50,
                                       od_type='Iter', 
                                       random_state=10, 
                                       thread_count=8, 
                                       l2_leaf_reg=1, 
                                       task_type='GPU',
                                       verbose=False)
    model_cat.fit(X_train[i], y_train[i], eval_set=[(X_valid[i], y_valid[i])], early_stopping_rounds=50, use_best_model=True)
    # print(model_cat.evals_result_)
    print(model_cat.best_score_['validation']['AUC'])


============================CAT training use Data 1/5============================

0.6634469628334045

============================CAT training use Data 2/5============================

0.6711283624172211

============================CAT training use Data 3/5============================

0.6696160435676575

============================CAT training use Data 4/5============================

0.6644403040409088

============================CAT training use Data 5/5============================

0.6695903539657593
CPU times: user 1min 33s, sys: 1min 13s, total: 2min 47s
Wall time: 1min 35s


In [ ]:
# catgbm模型
# lr=0.1, iterations=500, max_depth=12
%%time
pred_cats = []
for i in range(5):
    print('\n============================CAT training use Data {}/5============================\n'.format(i+1))
    model_cat = cat.CatBoostClassifier(learning_rate=0.1, 
                                       iterations=500, 
                                       max_depth=10,
                                       eval_metric='AUC', 
                                       od_wait=50,
                                       od_type='Iter', 
                                       random_state=10, 
                                       thread_count=8, 
                                       l2_leaf_reg=1, 
                                       task_type='GPU',
                                       verbose=False)
    model_cat.fit(X_train[i], y_train[i], eval_set=[(X_valid[i], y_valid[i])], early_stopping_rounds=50, use_best_model=True)
    # print(model_cat.evals_result_)
    print(model_cat.best_score_['validation']['AUC'])


============================CAT training use Data 1/5============================

0.6793952286243439

============================CAT training use Data 2/5============================

0.6837489604949951

============================CAT training use Data 3/5============================

0.6802981793880463

============================CAT training use Data 4/5============================

0.6703081130981445

============================CAT training use Data 5/5============================

0.6788195371627808
CPU times: user 1min 11s, sys: 42.3 s, total: 1min 54s
Wall time: 60 s


In [ ]:
# catgbm模型
# lr=0.1, iterations=1000, max_depth=12
%%time
pred_cats = []
for i in range(5):
    print('\n============================CAT training use Data {}/5============================\n'.format(i+1))
    model_cat = cat.CatBoostClassifier(learning_rate=0.1, 
                                       iterations=1000, 
                                       max_depth=10,
                                       eval_metric='AUC', 
                                       od_wait=50,
                                       od_type='Iter', 
                                       random_state=10, 
                                       thread_count=8, 
                                       l2_leaf_reg=1, 
                                       task_type='GPU',
                                       verbose=False)
    model_cat.fit(X_train[i], y_train[i], eval_set=[(X_valid[i], y_valid[i])], early_stopping_rounds=50, use_best_model=True)
    # print(model_cat.evals_result_)
    print(model_cat.best_score_['validation']['AUC'])


============================CAT training use Data 1/5============================

0.6793952286243439

============================CAT training use Data 2/5============================

0.6837489604949951

============================CAT training use Data 3/5============================

0.6802982091903687

============================CAT training use Data 4/5============================

0.6703081727027893

============================CAT training use Data 5/5============================

0.6788195371627808
CPU times: user 1min 11s, sys: 42.4 s, total: 1min 53s
Wall time: 59.8 s


In [ ]:
# catgbm模型
# lr=0.1, iterations=1500, max_depth=12
%%time
pred_cats = []
for i in range(5):
    print('\n============================CAT training use Data {}/5============================\n'.format(i+1))
    model_cat = cat.CatBoostClassifier(learning_rate=0.1, 
                                       iterations=1500, 
                                       max_depth=10,
                                       eval_metric='AUC', 
                                       od_wait=50,
                                       od_type='Iter', 
                                       random_state=10, 
                                       thread_count=8, 
                                       l2_leaf_reg=1, 
                                       task_type='GPU',
                                       verbose=False)
    model_cat.fit(X_train[i], y_train[i], eval_set=[(X_valid[i], y_valid[i])], early_stopping_rounds=50, use_best_model=True)
    # print(model_cat.evals_result_)
    print(model_cat.best_score_['validation']['AUC'])


============================CAT training use Data 1/5============================

0.6793952286243439

============================CAT training use Data 2/5============================

0.6837489604949951

============================CAT training use Data 3/5============================

0.6802981793880463

============================CAT training use Data 4/5============================

0.6703081130981445

============================CAT training use Data 5/5============================

0.6788195371627808
CPU times: user 1min 11s, sys: 42.5 s, total: 1min 53s
Wall time: 59.9 s
